Define your variables (That's the only cells you need to change). Then run the whole notebook

In [14]:
image_path = '/home/tli_project/Desktop/Data/T4/stage1/220125_analysis/220125_P36.5_N1/meta/N2V_220125_P36.5_N1_Crop1_Regi_GFP.tif'
save_path = '/home/tli_project/Desktop/Data/T4/stage1/220125_analysis/220125_P36.5_N1/meta/'
save_name = '220125_P36.5_N1_Crop1_Regi_GFP.tif'

#N2V parameters
model_name ='n2v_3D_v6'
model_path ='/home/tli_project/Desktop/TLI_preprocessing/Work_folder/N2V/models/'

#Clahe parameters:
clipLimit = 1
kernel_size = (60, 60)

In [15]:
if save_path[-1] != '/':
    save_path += '/'

Load needed packages

In [16]:
import tifffile as tif
import numpy as np
import utils.denoise as denoise
import utils.datautils as datautils
import Neurosetta as neu
import matplotlib.pyplot as plt

In [17]:
# Read 4D GFP image
image_4D = tif.imread(image_path)

In [5]:
image_4D = denoise.N2V_4D(image_4D, model_name=model_name, model_path=model_path, 
                            save=True, save_path=save_path, xy_pixel=0.076, 
                            z_pixel=0.4, save_file=save_name) 

2022-06-14 15:09:35.143788: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-14 15:09:35.144553: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tli_project/miniconda3/envs/preprocessing/lib/python3.8/site-packages/cv2/../../lib64:/app/lib
2022-06-14 15:09:35.144571: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-14 15:09:35.144598: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: l-p0-345
2022-06-14 15:09:35.144603: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: l-p0-345
2022-06-14 15:09:35.144758: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find lib

Loading network weights from 'weights_best.h5'.


applying N2V:   0%|          | 0/91 [00:00<?, ?it/s]

The input image is of type uint16 and will be casted to float32 for prediction.


2022-06-14 15:09:37.467569: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-14 15:09:37.492862: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2496000000 Hz
applying N2V:   1%|          | 1/91 [01:55<2:53:21, 115.57s/it]

...

applying N2V:  98%|█████████▊| 89/91 [2:21:03<03:06, 93.29s/it]

The input image is of type uint16 and will be casted to float32 for prediction.


applying N2V:  99%|█████████▉| 90/91 [2:22:36<01:33, 93.19s/it]

The input image is of type uint16 and will be casted to float32 for prediction.


applying N2V: 100%|██████████| 91/91 [2:24:09<00:00, 95.05s/it]


In [18]:
# remove further noise pixels before applying clahe, by removing 95% quantil
# threshold = np.quantile(image_4D,0.95)
# # threshold = 4 * image_4D.mean()
threshold = 300
image_4D[image_4D<threshold] = 0
# image_4D[mask == 0] = 0

In [19]:
image_4D.min(), image_4D.max()

(0, 2613)

In [20]:
image_4D = denoise.clahe_4D(image_4D, kernel_size=kernel_size, clipLimit=clipLimit, 
                            save=True, save_path=save_path, xy_pixel=0.076, 
                            z_pixel=0.4, save_file=save_name)

applying clahe: 100%|██████████| 91/91 [04:03<00:00,  2.68s/it]


In [ ]:
# tif.imwrite(save_path+save_name+'1', image_4D, bigtiff=True, resolution=(1./1, 1./1),
#                 metadata={'spacing': 1, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

In [7]:
mask = denoise.mask_4D(image_4D, return_mask=False, 
                        xy_pixel=0.076, z_pixel=0.4, 
                        sig=1, save=True, 
                        save_path=save_path, 
                        save_file=save_name)

masking the image: 100%|██████████| 93/93 [03:20<00:00,  2.15s/it]
/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning: <tifffile.TiffWriter 'mask_SyNRA_4D_211114_P36.tif'> truncating ImageJ file
  warnings.warn(


In [14]:
mask.min(), mask.max(), mask.mean(), np.quantile(mask, 0.999)

(0, 54287, 117.13803814904556, 28049.0)

In [8]:
mask[mask>0] = 1
image_4D = image_4D * mask
save_file = save_path+save_name+'_masked.tif'
tif.imwrite(save_file, image_4D, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning: <tifffile.TiffWriter 'SyNRA_4D_211114_P36_masked.tif'> truncating ImageJ file
  warnings.warn(


In [15]:
# # masking the image using neurosetta
# image_4D = neu.Neuron(image_4D)
# mask =  neu.mask_Neuron(image_4D)
# image_4D = image_4D.array
# mask = mask.array
# for i, img in enumerate(image_4D):
#     image_4D[i] = datautils.img_limits(mask[i], limit=img.max())
# image_4D = image_4D.astype('int16')
# datautils.save_image(save_path+'mask2_'+save_name, image_4D)

In [16]:
print('memory cleared, and kernel stopped')
print('Job is DONE!')
os._exit(00)

Error: Canceled future for execute_request message before replies were done